# Introduction to track reconstruction using quantum algorithms at LUXE

This is an introduction about the concepts of track reconstruction at LUXE.
This notebook will provide you with information and tasks to consolidate your knowledge.


If something is unclear or if you just want to discuss something, don't forget: 

**We are always happy to answer your questions and to help you at any stage of the project!**

## Checklist before starting to work on this noteboook

If you feel, that at least one of the following statements is not true, let's schedule a brief zoom meeting to make them true :)

* I have knowledge about how charged particles are influenced by an magnetic field (bending raidus)
* I have knowledge about the following python features:
    - importing amd using (selfwritten) modules
    - classes in python, in particular using and accessing functions and attributes/fields in classs
    - plotting data with matplotlib

##  I. Tracking data
    a) LUXE positron tracking system
    b) hits and tracks
    c) file structure
    d) load and process tracking data

## II. Forming X-plets
    a) a Combinatorial problem 
    b) doublets and triplets
          - task 1: combinatorics
    c) preselection at doublet level 
    d) creating doublets: an example
          - task 2a) : preselection on doublet level - Finding dx/x0 and epsilon
          - task 2b) : testing preselection parameters hypothesis
          - task 2c) : finding a good angular condition - triplet preselection
    e) creating triplets: an example

## III. The (almost) real thing

## I. Tracking data
### a) LUXE positron tracking system

The LUXE electron-positron are created by shooting a powerful laser at the European XFEL 16.5 GeV electron beam. With a strong magnet these electron-positron pairs are separated and guiding to their respective tracking systems and calorimeters. 

In the following figure, the location of the positron tracking system is marked with the red arrow. The positron tracking system consists of :
* 8 partly overlapping staves (green rectangles) and contain 
* 9 chips with 1024 x 512 pixels each and 
* a pixel size of 27$\mu m$ x 29$\mu m$. 

<img src="img/Positron_Tracking_System.png" width=600 height=600/>

For the track reconstruction task we start with a simplified simulation. With this tool we model the deflection of the positrons through the magnetic field and the interaction of the positrons with the detector layer (scattering e.g.). 

We are using a simplified simulation and not Geant4 (only at the moment) because we can tune and switch on/off these processes and also simplify the detector geometry to better understand the impact on the track reconstruction algorithm. 

The initial distribution and number of the positrons to use in our simplified simulation are taken from MC simulations with ptarmigan, which models what's hapening at the IP at LUXE. You do not need to understand what's happening in the frame of ptarmigan. But in case you are interested in this, you can have a look at https://arxiv.org/pdf/2108.10883.pdf.

The output of the simplified simulation with the LUXE positron tracker geometry as it is (~10/2021) is displayed in the following figure. Hits on the detector layers are marked as red, whereas the green lines are drawn track examples.

You may notice the darker shades of grey. These are actually very small gaps between the chips on each stave. These gaps would make track reconstruction a bit more challenging.

<img src="img/Luxe_FL.png" width=600 height=600/>

Overlapping + gaps between chips on the staves are the reasons why we are using a simplified geometry to make the track reconstruction task not unnessessarily complicated. We're aiming at getting knowledge about how to set parameters etc. of our track reconstruction algorithm at this simplier case. As soon if we're confident about our modelling, we will come back to the actual LUXE geometry.

As a summary, in our simplified simulation:
* we neglect that placing multiple chips on a stave results in small gaps between the chips
* we assume that we have only four layers without any overlaps 

Then the output of the simplified simulation then changes to:

<img src="img/Luxe_SL.png" width=600 height=600/>

### b) Hits and tracks




Reading out the pixel information of a detector as "1 activated pixel = 1 hit from a particle" is unfortunately not correct. A particle can trigger more than one pixel, so before extracting the position information a hit-clustering has to be done. A nice example of how particles might create single and combined clusters is given in the following figure, taken from https://arxiv.org/pdf/1704.07983.pdf:

<img src="img/Hit_Clusters.png" width=600 height=600/>

We skip this clustering, because as a starting point we work with not digitized data. We assume that we have perfect knowledge about the hit coordinates of the particles on the detector.

When combining hits from different detector layers a track can be formed. For the simplified geometry we define a track as

**Four Hits From Consecutive Layers**.

### c) File structure

We have to save the information about the hits on the detector layer in a dedicated file format. A very easy and cross-platform accessible way is to use the .csv format.

This is why we agreed on a data structure for the hits on the detector layer, inspired by the "Track ML Challenge" dataset, which is available on kaggle for example. See more at https://www.kaggle.com/competitions/trackml-particle-identification/.

For every particle-detector interaction we save the following values:
- **hit_ID**: A unique integer number across all hits on all detector layers within an event
- **x**: x position of the hit in [m]
- **y**: y position of the hit in [m]
- **z**: z position of the hit in [m]
- **layer**: detector layer identifier, they are labeled as Planes

Also available are the following truth information, meaning simulation only:
- **particle_ID**: integer number, identifying the particle which created the hit    
- **particle_energy**: energy of the particle in [MeV]

The "tracking.csv" file will look like this then:

<img src="img/CSV_File_Example.png" width=600 height=600/>

The file shown above has ~40000 entries, which correspond to ~10000 particles, because each particle will interact with each of the detector layers one time. Particles that are leaving the detector range due to multiple scattering will produce less than four entries in the .csv file.

### d) Load and process tracking data

As an example, we prepared a file with just 3 particles, to show how we process these data. We will only use 'hit_ID', 'layer', 'x', 'y' and 'z' for creating track candidates. Information on 'particle_ID' is only used for performance measurements, since this is an simulation exclusive value.

A smart way to read in the tracking data from file is to use pandas. It is very good a grouping, slicing and presenting tabular data. 
For very big data sets it is better to use the csv module of python, because the many and nice features of pandas come with a higher cost in computational time and memory. Two examples are given in the following.

In [ ]:
import pandas as pd
import csv

In [ ]:
# load data with pandas
tracking_data = pd.read_csv("examples/ExampleTrackingData.txt")
print(tracking_data)

In [ ]:
# load data with csv and context manager

csv_data = []
with open("examples/ExampleTrackingData.txt") as file:
    csv_reader = csv.reader(file)
    csv_header = next(csv_reader)                # access header, csv files should consist of one line of header
    for row in csv_reader:                       # iterate over all rows in the csv file
        csv_data.append(row)

print(csv_header)
for data in csv_data:
    print(data)
    

# The '' signs are indicating that the values are strings.
# To work with this data one has to convert the strings for x,y and z later! 

## II. Forming X-plets
### a) A Combinatorial problem 

The term "x-plets" stands for a set of connected hits on different, consecutive detector layers. X-plets are basically parts of track candidates. Our goal is to find an optimal way of constructing and connecting x-plets to tracks with respect to some objective function.

Let's first let's have a look at combinatorics:
In the following figure a graphical representation of the example file from before is given. Asumming 3 particles and 4 layers we will end up with 12 hits. These hits are represented by the red dots. Our beam/forward direction is the z-axis. For now we stick to a 2D-representation of the example:

<img src="img/Raw_Hits_Detector.png" width=600 height=600/>

### b) Doublets and Triplets

We start with a very basic assumption and state that 

**Two hits of the same detector layer cannot form an x-plet**.

Also it is: 

**Forbidden to "jump" over a detector layer in the process of creating doublets and triplets**. 

So, for example a doublet from hits of detector layer 1 and 3 is not allowed. 
We also define, that a

- **Doublet** is a set of two hits of consecutive layers.
- **Triplet** is a set of two doublets with exactly one shared hit 
- **Track Candidate** is a set of two triplets with exactly two shared hits on the two middle layers

Example how forming doublets, triplets and and track candidates could look like:

<img src="img/Doublet_Triplet_Track.png" width=600 height=600/>

### Task I: Combinatorics

Please consider the example above and answer the following questions and derive an equation with respect to the number of involved particles:
- How many doublets can be created?
- How many triplets can be created?
- How many track candidates can be created?
- We expect up to 1 Million positrons at LUXE. How many doublets, triplets and track candidates would we have to create?


### c) Preselection at doublet level

A brute force approach is computational not feasible for a high number of particles. That's why we need to create doublets and triplets in a smart way. So let's have a look at the physics first:

Within the detector area there is no magnetic field. But the tracks of the positrons are bent before by a dipole magnet.
That means that tracks from positrons will form an angle with the beamaxis z with respect to their initial energy. A possible scenario could look like this:

<img src="img/Energy_Selection.png" width=600 height=600/>

For a particle of mass **m**, Energy **E** and momentum **p** in a uniform B-field, the bending radius of the motion of the particle **inside the dipole field** is given by:

$\frac{1}{\rho}= \frac{e\cdot B}{p \cdot c} = \frac{e\cdot B}{\beta E} $

This means, that a higher energy will lead to less bending, and a higher magnetic field strength will bend the particle trajectory more. For our experimental setup we can conclude, that - in general - hits with a **higher x value** on the first detector layer have a **lower energy**, due to more bending of the particle trajectory. 

We now want to look for a connection between hits of one particle. Recalling the figure above one can find two assumptions: 

- The difference in x-values between **two consecutive hits** on different detector layers of **the same particle** decreases with the particle energy
- Hits on the first detector layer with a **lower x value** have a **higher energy**

Not all doublets which can be created from the hits have a hit on the first detector. But we want to include information about where this doublet - which is a part of track - would have a hit on the first detector layer. 

We model the information about the actual hit on the first detector layer - if a doublet was created from layer 1 -> 2 or an expected hit - if the doublet has no hit on detector layer 1 -  on the first detector layer from extrapolation into the modelling of our equation. We name the x-value on the first detector layer "x0".

Every doublet that is worth considering for a piece of a track has to then to fulfill the equation:

$\frac{dx}{x_0} \approx c = const$

with $dx= x_2 - x_1$ and $x_0$ as shown in the following figure:


<img src="img/dx_x0_criteria.png" width=600 height=600/>

In an ideal case $\frac{dx}{x_0}$ would be very close to a constant value. But, there is also scattering, so the final equation needs to point at an interval and not a value:

$\frac{dx}{x_0} \in [c - \epsilon, c + \epsilon]$ 

### c) creating doublets: an example

Now we want to give you an example on how this is actually done, when we are using it for our needs.
To handle our doublets as an object, we wrote a class named Doublet. 
A doublet object hast 6 attributes:
- **hit_1_particle_key**: $\hspace{0.09cm}$ the particle ID of the first hit, given by the "particle_ID" information in the .csv file
- **hit_1_position**: $\hspace{0.8cm}$ the particle position of the first hit as an array [x, y, z], given by the "x", "y" and "z" information in the .csv file
- **hit_2_particle_key**: $\hspace{0.09cm}$ the particle ID of the second hit, given by the "particle_ID" information in the .csv file
- **hit_2_position**: $\hspace{0.8cm}$ the particle position of the second hit as an array [x, y, z], given by the "x", "y" and "z" information in the .csv file
- **hit_1_id**: $\hspace{1.9cm}$ the hit ID of the first hit, given by the "hit_ID" information in the .csv file
- **hit_2_id**: $\hspace{1.9cm}$ the hit ID the second hit, given by the "hit_ID" information in the .csv file

To use the class we first have to import it of course.

In [ ]:
from doublet import Doublet

For creating a doublet, you just have to provide the parameters given above. Every information you need is stored inside the .csv file. You just need to take care of the variable types since they are all strings! An example is shown below:

In [ ]:
x1 = float('1.0')          # x = '1.0' in .csv file. y, z accordingly 
y1 = float('3.0')
z1 = float('1.0')

x2 = float('2.0')
y2 = float('4.0')
z2 = float('1.0')

particle_id_1 = "1"        # no adjustment needed for particle and hit id
particle_id_2 = "2"

hit_id_1 = "5678"
hit_id_2 = "8765"


test_doublet = Doublet(particle_id_1,
                       [x1, y1, z1],
                       particle_id_2,
                       [x2, y2, z2],
                       hit_id_1,
                       hit_id_2)
print(test_doublet)

Now we can encode our data from the .csv file into doublets.

First we order the data with respect to the z-value of the hits, meaining we order them by layer.
We know, that the detector layers have z-values of 0.0, 1.0, 2.0 and 3.0. 

In [ ]:
# Recall what the data set looked like. 
csv_data

In [ ]:
# entry[3] is the z-value of the hits. We're doing this sorting taks with list comprehension
csv_data_detector_layer_0 = [entry for entry in csv_data if entry[3] == '0.0']
csv_data_detector_layer_1 = [entry for entry in csv_data if entry[3] == '1.0']
csv_data_detector_layer_2 = [entry for entry in csv_data if entry[3] == '2.0']
csv_data_detector_layer_3 = [entry for entry in csv_data if entry[3] == '3.0']

In [ ]:
# check if lists now only contain data from one layer. If only entries with 'Plane 0' for csv_data_detector_layer_0 
# what we did has worked
csv_data_detector_layer_0

Now we can create the doublets. There will be 3 lists of doublets:
- doublet_list01: hits on detector layer 0 connected with hits on detector layer 1
- doublet_list12: hits on detector layer 1 connected with hits on detector layer 2
- doublet_list23: hits on detector layer 2 connected hits hits on detector layer 3

In [ ]:
# define empty lists 
doublet_list01 = []
doublet_list12 = []
doublet_list23 = []

In [ ]:
# hit[0]: hit_ID 
# hit[1]: x 
# hit[2]: y 
# hit[3]: z 
# hit[5]: particle_ID    
# need to convert strings for x, y and z to float for later calculations
# Filling the lists:
for hit_1 in csv_data_detector_layer_0:        # loop over first detector layer
    for hit_2 in csv_data_detector_layer_1:    # loop over second detector layer
        doublet_list01.append(Doublet(hit_1[5],                                            # particle ID first hit
                                      [float(hit_1[1]), float(hit_1[2]), float(hit_1[3])], # [x, y, z] first hit
                                      hit_2[5],                                            # particle ID second hit
                                      [float(hit_2[1]), float(hit_2[2]), float(hit_2[3])], # [x, y, z] second hit
                                      hit_1[0],                                            # hit ID first hit
                                      hit_2[0]))                                           # hit ID second hit
for hit_1 in csv_data_detector_layer_1:        # loop over second detector layer
    for hit_2 in csv_data_detector_layer_2:    # loop over third detector layer
        doublet_list12.append(Doublet(hit_1[5],                                            # particle ID first hit
                                      [float(hit_1[1]), float(hit_1[2]), float(hit_1[3])], # [x, y, z] first hit
                                      hit_2[5],                                            # particle ID second hit
                                      [float(hit_2[1]), float(hit_2[2]), float(hit_2[3])], # [x, y, z] second hit
                                      hit_1[0],                                            # hit ID first hit
                                      hit_2[0]))   
for hit_1 in csv_data_detector_layer_2:        # loop over third detector layer
    for hit_2 in csv_data_detector_layer_3:    # loop over foruth detector layer
        doublet_list23.append(Doublet(hit_1[5],                                            # particle ID first hit
                                      [float(hit_1[1]), float(hit_1[2]), float(hit_1[3])], # [x, y, z] first hit
                                      hit_2[5],                                            # particle ID second hit
                                      [float(hit_2[1]), float(hit_2[2]), float(hit_2[3])], # [x, y, z] second hit
                                      hit_1[0],                                            # hit ID first hit
                                      hit_2[0]))   


### Task 2a) : Preselection on doublet level - Finding dx/x0 and epsilon

Please make some plots of the distributions of $\frac{dx}{x_0}$ for all doublets.
For every doublet d in the list you can access the coordinates [x,y,z] by calling:

    d.hit_2_position and d.hit_1_position

You will get the value $x_0$ at agiven z by calling:
    
    d.x0_at_z(x_2, x1, z2, z1, z0)
    
You can also check if the doublet is stemming from one single particle by calling:

    d.is_correct_match

Can you find an interval where only correct doublets(passing d.is_correct_match) are located?

### Task 2b) : Testing preselection parameters hypothesis

In the following cell you have to choose the correct parameters for dx_x0 and dx_x0_epsilon to skim the list of (mostly unnessarily created) doublets. 
- If you already identified a good interval in the task before, then set the values accordingly. Be careful to keep all the correct ones! If you end up with 3 correct triplets and 3 or 4 kept doublets for each of the doublet lists, then you did well!

- Could you find an intervall which works for all the lists? Is it possible to get rid of all fake doublets by this procedure? What effect is limiting this preselection procedure?

In [ ]:
# Set values (--> task 2b)  
dx_x0 = 0.0
dx_x0_epsilon = 7

# this one is fixed for our example, don't touch it please!
z_reference = 0.0

In [ ]:
# Define a new list and save the doublets into this list if they pass the check

doublet_list01_skimmed = []
doublet_list12_skimmed = []
doublet_list23_skimmed = []

for doublet in doublet_list01:
    if doublet.dx_x0_check(dx_x0, dx_x0_epsilon, z_reference):   # if check passes, doublet is appended to the skimmed list
        doublet_list01_skimmed.append(doublet)  
for doublet in doublet_list12:
    if doublet.dx_x0_check(dx_x0, dx_x0_epsilon, z_reference):   # if check passes, doublet is appended to the skimmed list
        doublet_list12_skimmed.append(doublet)
for doublet in doublet_list23:
    if doublet.dx_x0_check(dx_x0, dx_x0_epsilon, z_reference):   # if check passes, doublet is appended to the skimmed list
        doublet_list23_skimmed.append(doublet) 

Let's check if it has worked:

In [ ]:
# First we evaluate the brute-force method
correct_kept_doublets_bf = 0
wrong_kept_doublets_bf = 0

for d in doublet_list01:
    if d.is_correct_match:
        correct_kept_doublets_bf += 1
    else:
        wrong_kept_doublets_bf += 1

print("Brute-Force creation of all doublets")
print(f"Number of correct doublets: {correct_kept_doublets_bf}")
print(f"Number of all doublets: {correct_kept_doublets_bf + wrong_kept_doublets_bf}")

In [ ]:
# Now we evaluate the skimmed list with our dx_x0 criteria applied
correct_kept_doublets = 0
wrong_kept_doublets = 0

for d in doublet_list01_skimmed:
    if d.is_correct_match:
        correct_kept_doublets += 1
    else:
        wrong_kept_doublets += 1
print("After removing unlikely combinations:")
print(f"Number of correct doublets: {correct_kept_doublets}")
print(f"Number of all doublets: {correct_kept_doublets + wrong_kept_doublets}")

### e) creating triplets: an example

Creating triplets out of doublets is much easier than creating doublets from hits. There are only two rules:
- a triplet consists of two connected doublets with one shared hit
- the angle in xz or yz is limited by some value

So creating triplet means, you loop over doublet lists and combine them if they are compatible. Therefore we need the triplet class:

In [ ]:
from triplet import Triplet

The triplet object consists of:

- **doublet_1**: a before created doublet 
- **doublet_2**: another before created doublet
- **triplet_id**: some unique identifyer, string or integer


### Task 2c) : Finding a good angular condition  - triplet preselection

The triplet class already provides you with a function to check how big the angle difference in xz and yz is.
you can do this by calling

    t.angles_between_doublets()
It returns [xz_angle, yz_angle] with xz_angle and yz_angle in rad.

Again, you can check with

    t.is_correct_match
    
if a triplet is stemming from one particle (correct match)

Please make a plot of the angle distributions of xz and find a good cut value. You can choose the value in the cell below to select the correct triplets.

In [ ]:
# Creating triplets from the first two doublet lists. We're now not adding the triplets to list if the
# cut value does not apply:
angular_condition_for_triplet_creation = 1.4

triplet_list_012 = []
id_counter = 0    # for giving the triplets a name
for doublet_1 in doublet_list01:                                                              # loop over doublet list 01
    for doublet_2 in doublet_list12:                                                          # loop over doublet list 12
        if doublet_1.hit_2_position == doublet_2.hit_1_position:                              # check if shared hit
            
            t = Triplet(doublet_1, 
                        doublet_2, 
                        id_counter)                                     # create triplet object
            
            if abs(t.angles_between_doublets()[0]) <  angular_condition_for_triplet_creation:       
                triplet_list_012.append(t)
        id_counter += 1 
        

Let's check the performance of your selection:

In [ ]:
# Now we evaluate the skimmed list with our dx_x0 criteria applied
correct_kept_triplets = 0
wrong_kept_triplets = 0

for t in triplet_list_012:
    if t.is_correct_match:
        correct_kept_triplets += 1
    else:
        wrong_kept_triplets += 1
print("After removing unlikely combinations:")
print(f"Number of correct triplets: {correct_kept_triplets}")
print(f"Number of all triplets: {correct_kept_triplets + wrong_kept_triplets}")

Great! We now have our triplets!

We use information which is storred inside the triplet objects for the quantum computing part of our track reconstruction. 

Many of the created triplets are "fake tracks" or at least parts of fake tracks. That means, that the detector hits which are stored inside the triplets, are not from the same particle. Since we do not have this truth knowledge for real data, we need an algorithm which tells us which triplets to keep and which to discard. And that's exactly where the quantum computing part is used.

## Checklist after finishing this noteboook

If you feel, that at least one of the following statements is not true, let's schedule a meeting to make them true :)

* I have a good overview about the LUXE positron detection system geometry
* I know how I can access the tracking data files, load, inspect and process them
* I am familiar with the concept of grouping hits to x-plets
* I know why it is important to start with a preselection instead of a brute force approach
* I can use the doublet and triplet class and create doublets and triplets from the given data
* I can identify the parameters for a successful preselection and perform it also

# III. The (almost) real thing

There's another file in the *examples/* folder. It contains data from our Simplified Simulation. Can you find a best preselection for it?
If you accept the challenge, please measure your results in:

* Number of participating particles / (0.5 * kept correct triplets)

which is some kind of preselection efficiency. Maybe you can also come up with your own idea to improve the preselection!